# 2.10 Proyecto Intermedio\n\n- Objetivo: integrar CTEs, window functions, segmentación y KPIs.\n- Prerrequisitos: Level02 2.1–2.9 completado.\n- Entregables: consultas consolidadas para informe ejecutivo.\n- Reto: vista única con métricas clave y controles de calidad.\n- Errores comunes: granularidad en RFM, olvidos de NULLIF, PARTITION en rankings.

## Requerimientos

1. Ranking de productos por margen acumulado y variación vs día anterior.
2. RFM simplificado de clientes.
3. Ingresos diarios combinados ventas + suscripciones y crecimiento.
4. Top segmento por margen y ticket promedio.
5. Detección de días sin ventas (calendario).

🟢 Paso 1: Ránking margen acumulado.

In [ ]:
WITH margen_prod AS (
  SELECT p.producto_id, p.nombre, fecha, SUM(fv.cantidad*(p.precio_unitario-p.costo_unitario)) AS margen_dia
  FROM dbo.fact_ventas fv
  JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
  GROUP BY p.producto_id, p.nombre, fecha
), acum AS (
  SELECT producto_id, nombre, fecha, margen_dia,
         SUM(margen_dia) OVER (PARTITION BY producto_id ORDER BY fecha ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS margen_acum,
         LAG(margen_dia) OVER (PARTITION BY producto_id ORDER BY fecha) AS margen_prev
  FROM margen_prod
)
SELECT *, CASE WHEN margen_prev IS NULL THEN NULL ELSE margen_dia - margen_prev END AS delta_margen
FROM acum;

🟠 Paso 2: RFM simplificado clientes.

In [ ]:
WITH actividad AS (
  SELECT c.cliente_id, MAX(fv.fecha) AS ultima_fecha, COUNT(DISTINCT fv.fecha) AS dias_con_venta,
         SUM(fv.cantidad*(p.precio_unitario-p.costo_unitario)) AS margen_total
  FROM dbo.fact_ventas fv
  JOIN dbo.dim_clientes c ON fv.cliente_id = c.cliente_id
  JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
  GROUP BY c.cliente_id
)
SELECT cliente_id, dias_con_venta, margen_total,
       DATEDIFF(DAY, ultima_fecha, CAST(GETDATE() AS DATE)) AS recencia_dias
FROM actividad;

🟠 Paso 3: Ingresos diarios combinados.

In [ ]:
WITH ventas AS (
  SELECT fecha, SUM(cantidad*p.precio_unitario*(1 - descuento_pct/100.0)) AS ingresos_ventas
  FROM dbo.fact_ventas fv
  JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
  GROUP BY fecha
), sus AS (
  SELECT fecha_inicio AS fecha, SUM(costo_mensual) AS ingresos_sus
  FROM dbo.fact_suscripciones
  GROUP BY fecha_inicio
), total AS (
  SELECT v.fecha, ingresos_ventas, ISNULL(s.ingresos_sus,0) AS ingresos_sus,
         (ingresos_ventas + ISNULL(s.ingresos_sus,0)) AS ingresos_total
  FROM ventas v LEFT JOIN sus s ON v.fecha = s.fecha
)
SELECT fecha, ingresos_total,
       LAG(ingresos_total) OVER (ORDER BY fecha) AS ingresos_prev,
       CASE WHEN LAG(ingresos_total) OVER (ORDER BY fecha) = 0 THEN NULL
            ELSE (ingresos_total - LAG(ingresos_total) OVER (ORDER BY fecha)) / NULLIF(LAG(ingresos_total) OVER (ORDER BY fecha),0) END AS pct_crecimiento
FROM total;

🔴 Paso 4: Segmento top por margen y ticket.

In [ ]:
WITH seg AS (
  SELECT c.segmento, SUM(fv.cantidad*(p.precio_unitario-p.costo_unitario)) AS margen_total,
         CAST(SUM(fv.cantidad*p.precio_unitario*(1 - fv.descuento_pct/100.0)) AS DECIMAL(10,2)) / COUNT(*) AS ticket_promedio
  FROM dbo.fact_ventas fv
  JOIN dbo.dim_clientes c ON fv.cliente_id = c.cliente_id
  JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
  GROUP BY c.segmento
)
SELECT *, RANK() OVER (ORDER BY margen_total DESC) AS rk_margen
FROM seg;

🔴 Paso 5: Días sin ventas (esquema conceptual).

In [ ]:
-- Requiere generación de serie de fechas (ver nivel previo).
-- WITH limites AS (SELECT MIN(fecha) AS fmin, MAX(fecha) AS fmax FROM dbo.fact_ventas),
-- serie AS (SELECT fmin AS fecha FROM limites UNION ALL SELECT DATEADD(DAY,1,fecha) FROM serie s JOIN limites l ON s.fecha < l.fmax)
-- SELECT fecha FROM serie EXCEPT SELECT DISTINCT fecha FROM dbo.fact_ventas;

⚙️ Desafío Senior: Consolidar todo en vista o script que exporte métricas clave ordenadas y con controles de calidad (anomalías).

Errores comunes: mezclar granularidades en RFM; no usar NULLIF en divisiones; ranking sin PARTITION cuando corresponde.

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.

---
## Navegación
[⬅️ Anterior](09_integracion_fuentes.ipynb)
---
